In [297]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np
class SimpleNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.c = nn.Sequential(nn.Linear(1, 2048), nn.Tanh(), nn.Linear(2048, 1)) 
    def forward(self, x):
        return self.c(x)
    
model = SimpleNN()
i = torch.arange(1, 5, 0.01, dtype=torch.float).unsqueeze(1)
test_i = torch.arange(5, 10, 0.01, dtype=torch.float).unsqueeze(1)

Right-hand-side of differential equation.

In [298]:
def f(x):
    return torch.pow(x, 2) + torch.exp(x)

This is actually a solution to differential equation using Wolfram Alpha calculator.


In [299]:
def anti_derivative_f(x):
    return torch.pow(x, 3) / 3 + torch.exp(-x) + torch.exp(x) / 2 - torch.pow(x, 2) + 2*x

In [300]:
def loss_function(output, target):
    return torch.abs(output - target)

In [301]:
test_f = f(test_i)
f = f(i)
f_anti_deriv = anti_derivative_f(i)

In [302]:
optimizer = torch.optim.Adam(model.parameters())

In [303]:
device='cuda'

In [ ]:
model.train()
model.to(device)
epochs = 100
for k in range(epochs):
    for j in range(len(i)):
        POINT = i[j].to(device)
        f_x_i = model(POINT)
        f_x_i_1 = model(POINT + 0.001)
        f_x_i_m1 = model(POINT - 0.001)
        grad = (f_x_i_1 - f_x_i) / 0.001 + (f_x_i_1 - 2 * f_x_i + f_x_i_m1) / ((0.001) ** 2)  # y' + y''
        loss = loss_function(grad, f[j].to(device))  # Error: y'(x_i) + y''(x_i) - f(x_i)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [ ]:
model.eval()
predictions = (model(i.to(device)).cpu().detach() + torch.tensor(400)).numpy()  # constant is important.
plt.plot(predictions, color='b')
plt.plot(f, color='r')
plt.plot(f_anti_deriv, color='y')#
#plt.plot(f_deriv, color='y')
plt.show()

In [ ]:
f_anti_deriv_test = anti_derivative_f(test_i)

In [ ]:
predictions = (model(test_i.to(device)).cpu().detach() + torch.tensor(600)).numpy()
plt.plot(predictions, color='b')
plt.plot(f_anti_deriv_test, color='y')#
plt.show() 

### Model can't approximate out of the range of seen data. The same I saw when approximating sin(x).